In [ ]:
from pyomo.environ import *
import json as js

# Function to load JSON data from a file
def load_json_data(filename):
    with open(filename, 'r') as file:
        return js.load(file)


# Model
model = ConcreteModel()

# Sets
teachers = {'Alice', 'Bob', 'Carol'}
courses = {'Math': 'Alice', 'Science': 'Bob', 'History': 'Carol'}
# classrooms = {'Room1': 30, 'Room2': 25}
data = load_json_data('Data/classrooms.json')
time_slots = {'Morning', 'Afternoon'}
enrollments = {'Math': 25, 'Science': 20, 'History': 30}

# Update to dynamically create classrooms set and parameters from JSON
classrooms_data = {classroom['code']: classroom['seats'] for classroom in data['classrooms']}
classrooms = classrooms_data.keys()

# Variables
model.schedule = Var(courses.keys(), classrooms.keys(), time_slots, within=Binary)

# Constraints
# 1. Each course is scheduled exactly once
def course_schedule_rule(model, c):
    return sum(model.schedule[c, r, t] for r in classrooms for t in time_slots) == 1
model.course_schedule = Constraint(courses.keys(), rule=course_schedule_rule)

# 2. Classroom conflict (No more than one course in each classroom at the same time slot)
def classroom_conflict_rule(model, r, t):
    return sum(model.schedule[c, r, t] for c in courses.keys()) <= 1
model.classroom_conflict = Constraint(classrooms, time_slots, rule=classroom_conflict_rule)

# 3. Classroom capacity
def classroom_capacity_rule(model, c, r, t):
    return enrollments[c] * model.schedule[c, r, t] <= classrooms_data[r]
model.classroom_capacity = Constraint(courses.keys(), classrooms, time_slots, rule=classroom_capacity_rule)

# Objective Function (dummy in this case, as we're only interested in feasibility)
model.obj = Objective(expr=1)

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# Display the solution
for c in courses.keys():
    for r in classrooms:
        for t in time_slots:
            if model.schedule[c, r, t].value == 1:
                print(f"Course {c} is scheduled in {r} during the {t} time slot.")

In [3]:
from pyomo.environ import *
import json as js

# Function to load JSON data from a file
def load_json_data(filename):
    with open(filename, 'r') as file:
        return js.load(file)

# Assuming you have already loaded the JSON data correctly
data = load_json_data('Data/classrooms.json')  # Make sure the path is correct

# Model
model = ConcreteModel()

# Sets
teachers = {'Alice', 'Bob', 'Carol'}
courses = {'Math': 'Alice', 'Science': 'Bob', 'History': 'Carol'}
time_slots = {'Morning', 'Afternoon'}
enrollments = {'Math': 25, 'Science': 20, 'History': 30}

# Update to dynamically create classrooms set and parameters from JSON
classrooms_data = {classroom['code']: classroom['seats'] for classroom in data['classrooms']}
classrooms = set(classrooms_data.keys())  # Ensure this is a set

# Variables
model.schedule = Var(courses.keys(), classrooms, time_slots, within=Binary)  # Use classrooms directly

# Constraints
# 1. Each course is scheduled exactly once
def course_schedule_rule(model, c):
    return sum(model.schedule[c, r, t] for r in classrooms for t in time_slots) == 1
model.course_schedule = Constraint(courses.keys(), rule=course_schedule_rule)

# 2. Classroom conflict (No more than one course in each classroom at the same time slot)
def classroom_conflict_rule(model, r, t):
    return sum(model.schedule[c, r, t] for c in courses.keys()) <= 1
model.classroom_conflict = Constraint(classrooms, time_slots, rule=classroom_conflict_rule)

# 3. Classroom capacity
def classroom_capacity_rule(model, c, r, t):
    return enrollments[c] * model.schedule[c, r, t] <= classrooms_data[r]
model.classroom_capacity = Constraint(courses.keys(), classrooms, time_slots, rule=classroom_capacity_rule)

# Objective Function (dummy in this case, as we're only interested in feasibility)
model.obj = Objective(expr=1)

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# Display the solution
for c in courses.keys():
    for r in classrooms:
        for t in time_slots:
            if model.schedule[c, r, t].value == 1:
                print(f"Course {c} is scheduled in {r} during the {t} time slot.")


Course Math is scheduled in CR02 during the Afternoon time slot.
Course Science is scheduled in CR20 during the Morning time slot.
Course History is scheduled in CR19 during the Morning time slot.


In [2]:
from pyomo.environ import *
import json

# Function to load JSON data from a file
def load_json_data(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Assuming you have already loaded the JSON data correctly
# Please replace 'Data/classrooms.json' with the correct path to your JSON file
data = load_json_data('Data/classrooms.json')

# Model initialization
model = ConcreteModel()

# Sets
teachers = {'Alice', 'Bob', 'Carol'}
courses = {'Math': 'Alice', 'Science': 'Bob', 'History': 'Carol'}
time_slots = {'Morning', 'Afternoon'}
enrollments = {'Math': 25, 'Science': 20, 'History': 30}

# Updating classrooms data from JSON
classrooms_data = {classroom['code']: classroom['seats'] for classroom in data['classrooms']}
classrooms = set(classrooms_data.keys())

# Mapping for classroom names
classroom_names = {classroom['code']: classroom['name'] for classroom in data['classrooms']}

# Variables
model.schedule = Var(courses.keys(), classrooms, time_slots, within=Binary)

# Constraints
# 1. Each course is scheduled exactly once
def course_schedule_rule(model, c):
    return sum(model.schedule[c, r, t] for r in classrooms for t in time_slots) == 1
model.course_schedule = Constraint(courses.keys(), rule=course_schedule_rule)

# 2. Classroom conflict (No more than one course in each classroom at the same time slot)
def classroom_conflict_rule(model, r, t):
    return sum(model.schedule[c, r, t] for c in courses.keys()) <= 1
model.classroom_conflict = Constraint(classrooms, time_slots, rule=classroom_conflict_rule)

# 3. Classroom capacity
def classroom_capacity_rule(model, c, r, t):
    return enrollments[c] * model.schedule[c, r, t] <= classrooms_data[r]
model.classroom_capacity = Constraint(courses.keys(), classrooms, time_slots, rule=classroom_capacity_rule)

# Objective function (assuming feasibility check)
model.obj = Objective(expr=1)

# Solver
solver = SolverFactory('glpk')
solver.solve(model)

# Display the solution with classroom names
for c in courses.keys():
    for r in classrooms:
        for t in time_slots:
            if model.schedule[c, r, t].value == 1:
                classroom_name = classroom_names[r]  # Fetching the classroom name using the code
                print(f"Course {c} is scheduled in {classroom_name} during the {t} time slot.")
 

Course Math is scheduled in Art Classroom during the Afternoon time slot.
Course Science is scheduled in Physics Lab during the Morning time slot.
Course History is scheduled in Biology Lab during the Afternoon time slot.


In [15]:
from pyomo.environ import *
import json

# Function to load JSON data from a file
def load_json_data(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Load classroom data
classroom_data = load_json_data('Data/classrooms.json')

# Load custom time slots data
time_slots_data = load_json_data('Data/time_slots.json')

# Model initialization
model = ConcreteModel()

# Sets
teachers = {'Alice', 'Bob', 'Carol'}
courses = {'Math': 'Alice', 'Science': 'Bob', 'History': 'Carol'}
enrollments = {'Math': 25, 'Science': 20, 'History': 30}

# Updating classrooms data from JSON
classrooms_data = {classroom['code']: classroom['seats'] for classroom in classroom_data['classrooms']}
classrooms = set(classrooms_data.keys())

# Updating time slots data from JSON
time_slots = {slot['id'] for slot in time_slots_data['time_slots']}

# Mapping for classroom names
classroom_names = {classroom['code']: classroom['name'] for classroom in classroom_data['classrooms']}

# Mapping for time slot periods
time_slot_periods = {slot['id']: slot['period'] for slot in time_slots_data['time_slots']}

# Variables
model.schedule = Var(courses.keys(), classrooms, time_slots, within=Binary)

# Constraints
# 1. Each course is scheduled exactly once
def course_schedule_rule(model, c):
    return sum(model.schedule[c, r, t] for r in classrooms for t in time_slots) == 1
model.course_schedule = Constraint(courses.keys(), rule=course_schedule_rule)

# 2. Classroom conflict (No more than one course in each classroom at the same time slot)
def classroom_conflict_rule(model, r, t):
    return sum(model.schedule[c, r, t] for c in courses.keys()) <= 1
model.classroom_conflict = Constraint(classrooms, time_slots, rule=classroom_conflict_rule)

# 3. Classroom capacity
def classroom_capacity_rule(model, c, r, t):
    return enrollments[c] * model.schedule[c, r, t] <= classrooms_data[r]
model.classroom_capacity = Constraint(courses.keys(), classrooms, time_slots, rule=classroom_capacity_rule)

# Objective function (assuming feasibility check)
model.obj = Objective(expr=1)

# Solver
solver = SolverFactory('glpk')
solver.solve(model)

# Display the solution with classroom names and time slot periods
for c in courses.keys():
    for r in classrooms:
        for t in time_slots:
            if model.schedule[c, r, t].value == 1:
                classroom_name = classroom_names[r]  # Fetching the classroom name using the code
                time_slot_period = time_slot_periods[t]  # Fetching the period using the time slot ID
                print(f"Course {c} is scheduled in {classroom_name} during the {time_slot_period} time slot.")


Course Math is scheduled in Art Classroom during the Monday Morning 09-11 time slot.
Course Science is scheduled in Physics Lab during the Wednesday Morning 09-11 time slot.
Course History is scheduled in Biology Lab during the Monday Morning 09-11 time slot.


In [14]:
from pyomo.environ import *
import json

# Function to load JSON data from a file
def load_json_data(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Load existing data
classroom_data = load_json_data('Data/classrooms.json')
time_slots_data = load_json_data('Data/time_slots.json')

# Load new JSON data
lessons_data = load_json_data('Data/lessons.json')
teachers_data = load_json_data('Data/teachers.json')
cohorts_data = load_json_data('Data/cohorts.json')

# Initialize the model
model = ConcreteModel()

# Classrooms and time slots setup
classrooms_data = {classroom['code']: classroom['seats'] for classroom in classroom_data['classrooms']}
classrooms = set(classrooms_data.keys())
time_slots = {slot['id'] for slot in time_slots_data['time_slots']}

# Lessons data
lessons = {lesson['code']: {'name': lesson['name'], 'hours': lesson['hours'], 'classroom': lesson['classroom']} for lesson in lessons_data['lessons']}
all_lessons = set(lessons.keys())

# Teachers data
teachers = {teacher['name']: {'lessons': teacher['lessons'], 'unavailable_time_slots': teacher['unavailable_time_slots']} for teacher in teachers_data['teachers']}

# Cohorts data
cohorts = {cohort['name']: {'number_of_students': cohort['number_of_students'], 'lessons': cohort['lessons'], 'total_hours': cohort['total_hours']} for cohort in cohorts_data['cohorts']}

# Reverse mapping of lessons to teachers
lesson_to_teacher = {}
for teacher, details in teachers.items():
    for lesson in details['lessons']:
        lesson_to_teacher[lesson] = teacher

# After loading and preparing data structures, print summaries to verify correctness
# print(f"Loaded {len(classrooms)} classrooms, {len(time_slots)} time slots, {len(all_lessons)} lessons.")



# Variables
model.schedule = Var(all_lessons, classrooms, time_slots, within=Binary)

# Constraints
# Example constraint: Teacher availability
def teacher_availability_rule(model, c, r, t):
    if c in lesson_to_teacher:
        teacher = lesson_to_teacher[c]
        if t in teachers[teacher]['unavailable_time_slots']:
            return model.schedule[c, r, t] == 0
    return Constraint.Skip
model.teacher_availability = Constraint(all_lessons, classrooms, time_slots, rule=teacher_availability_rule)

# Objective function placeholder (needs to be defined based on specific goals)
model.obj = Objective(expr=1, sense=minimize)

# Solver setup and execution
solver = SolverFactory('glpk')
# Before solving, enable solver logging to get more insight
# solver.solve(model, tee=True)  # The tee=True option enables solver output
solver.solve(model)

# Example code snippet for displaying the scheduling results
print("\nScheduled Lessons:")
for c in all_lessons:
    for r in classrooms:
        for t in time_slots:
            # Ensure accessing the variable's value correctly
            if model.schedule[c, r, t].value is not None and model.schedule[c, r, t].value > 0.5:  # This checks if the schedule decision is true
                print(f"Lesson {c} is scheduled in {r} during {t}")



Scheduled Lessons:


In [5]:
from pyomo.environ import *
import json

# Function to load JSON data from a file
def load_json_data(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Load data
classroom_data = load_json_data('Data/classrooms.json')
time_slots_data = load_json_data('Data/time_slots.json')
lessons_data = load_json_data('Data/lessons.json')  # New
teachers_data = load_json_data('Data/teachers.json')  # New
cohorts_data = load_json_data('Data/cohorts.json')  # New

# Model initialization
model = ConcreteModel()

# Existing sets and data structures
classrooms_data = {classroom['code']: classroom['seats'] for classroom in classroom_data['classrooms']}
classrooms = set(classrooms_data.keys())
time_slots = {slot['id'] for slot in time_slots_data['time_slots']}
classroom_names = {classroom['code']: classroom['name'] for classroom in classroom_data['classrooms']}
time_slot_periods = {slot['id']: slot['period'] for slot in time_slots_data['time_slots']}

# New data structures
lessons = {lesson['code']: lesson for lesson in lessons_data['lessons']}
teachers = {teacher['name']: teacher for teacher in teachers_data['teachers']}
cohorts = {cohort['name']: cohort for cohort in cohorts_data['cohorts']}

# Update model sets to include lessons
model.lessons = Set(initialize=lessons.keys())
model.teachers = Set(initialize=teachers.keys())
model.cohorts = Set(initialize=cohorts.keys())

# Variables (updated to include lessons)
model.schedule = Var(model.lessons, classrooms, time_slots, within=Binary)

# Updating constraints and objective function to reflect the new structure will be necessary.
# For simplicity, here's an adapted version of one constraint:

# Example: A constraint ensuring that each lesson is scheduled at least once
def lesson_schedule_rule(model, l):
    return sum(model.schedule[l, r, t] for r in classrooms for t in time_slots) >= 1
model.lesson_schedule = Constraint(model.lessons, rule=lesson_schedule_rule)

def unique_lesson_schedule_constraint(model, r, t):
    return sum(model.schedule[l, r, t] for l in model.lessons) <= 1
model.unique_lesson_schedule = Constraint(classrooms, time_slots, rule=unique_lesson_schedule_constraint)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Step 1: Create a mapping of lessons to cohorts based on your JSON data
lessons_to_cohorts = {}
for cohort in cohorts_data['cohorts']:
    for lesson_code in cohort['lessons']:
        if lesson_code not in lessons_to_cohorts:
            lessons_to_cohorts[lesson_code] = []
        lessons_to_cohorts[lesson_code].append(cohort['name'])

# Step 2: Aggregate scheduled lessons for each cohort
cohort_schedules = {cohort['name']: {} for cohort in cohorts_data['cohorts']}
for l in model.lessons:
    for r in classrooms:
        for t in time_slots:
            if model.schedule[l, r, t].value == 1:
                lesson_name = lessons[l]['name']
                classroom_name = classroom_names[r]
                time_slot_period = time_slot_periods[t]
                for cohort_name in lessons_to_cohorts.get(l, []):
                    if t not in cohort_schedules[cohort_name]:
                        cohort_schedules[cohort_name][t] = []
                    cohort_schedules[cohort_name][t].append((lesson_name, classroom_name, time_slot_period))

# Step 3: Format and display the output for each cohort
for cohort_name, schedule in cohort_schedules.items():
    print(f"Cohort: {cohort_name}")
    for time_slot, lessons in sorted(schedule.items(), key=lambda x: x[0]):
        for lesson_detail in lessons:
            lesson_name, classroom_name, time_slot_period = lesson_detail
            print(f"{time_slot_period}: {lesson_name}: Classroom: {classroom_name}")
    print("\n" + "-"*40 + "\n")  # Separator for readability



Cohort: Future Innovators
Monday Morning 09-11: Foundational Mathematics: Classroom: Art Classroom
Thursday Morning 09-11: Software Development Lab: Classroom: Art Classroom
Thursday Morning 11-13: Introduction to Programming: Classroom: Art Classroom
Monday Afternoon 14-16: Computer Networks: Classroom: Art Classroom
Monday Afternoon 16-18: Cybersecurity Fundamentals: Classroom: Art Classroom
Wednesday Morning 09-11: Data Science with Python: Classroom: Art Classroom

----------------------------------------

Cohort: Art and Design Pioneers
Wednesday Afternoon 14-16: Contemporary Art Practices: Classroom: Art Classroom
Wednesday Afternoon 14-16: Architectural Design Fundamentals: Classroom: Fine Arts Studio
Wednesday Afternoon 16-18: Sculpture Techniques: Classroom: Fine Arts Studio
Thursday Afternoon 14-16: Digital Media Design: Classroom: Art Classroom
Friday Morning 09-11: Visual Communication: Classroom: Art Classroom
Monday Morning 11-13: 3D Art and Design: Classroom: Art Classro

In [8]:
from pyomo.environ import *
import json

# Function to load JSON data from a file
def load_json_data(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Load data
classroom_data = load_json_data('Data/classrooms.json')
time_slots_data = load_json_data('Data/time_slots.json')
lessons_data = load_json_data('Data/lessons.json')  # New
teachers_data = load_json_data('Data/teachers.json')  # New
cohorts_data = load_json_data('Data/cohorts.json')  # New

# Model initialization
model = ConcreteModel()

# Existing sets and data structures
classrooms_data = {classroom['code']: classroom['seats'] for classroom in classroom_data['classrooms']}
classrooms = set(classrooms_data.keys())
time_slots = {slot['id'] for slot in time_slots_data['time_slots']}
classroom_names = {classroom['code']: classroom['name'] for classroom in classroom_data['classrooms']}
time_slot_periods = {slot['id']: slot['period'] for slot in time_slots_data['time_slots']}

# New data structures
lessons = {lesson['code']: lesson for lesson in lessons_data['lessons']}
teachers = {teacher['name']: teacher for teacher in teachers_data['teachers']}
cohorts = {cohort['name']: cohort for cohort in cohorts_data['cohorts']}

# Update model sets to include lessons
model.lessons = Set(initialize=lessons.keys())
model.teachers = Set(initialize=teachers.keys())
model.cohorts = Set(initialize=cohorts.keys())

# Assuming each time slot represents 2 hours
time_slot_duration = 2

# Calculate required time slots for each lesson
required_time_slots = {lesson_code: lesson['hours'] // time_slot_duration 
                       for lesson_code, lesson in lessons.items()}


# Variables (updated to include lessons)
model.schedule = Var(model.lessons, classrooms, time_slots, within=Binary)
# This variable indicates if a lesson (l) is scheduled for a cohort (c) in a classroom (r) at a time slot (t)
model.cohort_schedule = Var(model.cohorts, model.lessons, classrooms, time_slots, within=Binary)

# Parameters
required_time_slots = {lesson['code']: lesson['hours'] // 2 for lesson in lessons_data['lessons']}  # Assuming 2 hours per time slot
lessons_to_cohorts = {lesson['code']: [] for lesson in lessons_data['lessons']}
for cohort in cohorts_data['cohorts']:
    for lesson_code in cohort['lessons']:
        lessons_to_cohorts[lesson_code].append(cohort['name'])

# --------------------------------
def update_cohort_schedules(model):
    for l in model.lessons:
        for c in model.cohorts:
            for r in classrooms:
                for t in time_slots:
                    # If lesson l is scheduled in classroom r at time slot t,
                    # and if cohort c is taking lesson l, then update model.cohort_schedule accordingly
                    if l in lessons_to_cohorts and c in lessons_to_cohorts[l]:
                        # This line is conceptual; actual implementation may vary
                        model.cohort_schedule[c, l, r, t].set_value(model.schedule[l, r, t].value)



def lesson_schedule_rule(model, l):
    return sum(model.schedule[l, r, t] for r in classrooms for t in time_slots) >= 1
model.lesson_schedule = Constraint(model.lessons, rule=lesson_schedule_rule)

def unique_lesson_schedule_constraint(model, r, t):
    return sum(model.schedule[l, r, t] for l in model.lessons) <= 1
model.unique_lesson_schedule = Constraint(classrooms, time_slots, rule=unique_lesson_schedule_constraint)

def lesson_time_requirement_rule(model, l):
    return sum(model.schedule[l, r, t] for r in classrooms for t in time_slots) == required_time_slots[l]
model.lesson_time_requirement = Constraint(model.lessons, rule=lesson_time_requirement_rule)

from pyomo.environ import Constraint, ConstraintList, Binary, ConcreteModel, Set, SolverFactory, Var

def correct_classroom_constraint(model, l, r, t):
    lesson_classroom = lessons[l]['classroom']
    # If the lesson requires a specific classroom, check it's scheduled in the correct one
    if lesson_classroom != "any":
        # If the lesson is assigned to an incorrect classroom, enforce the variable to be 0
        return model.schedule[l, r, t] == 0 if r != lesson_classroom else Constraint.Skip
    # If the lesson can be in any classroom, the constraint is always satisfied
    return Constraint.Skip

# Apply the corrected constraint to the model
model.correct_classroom_constraint = Constraint(model.lessons, classrooms, time_slots, rule=correct_classroom_constraint)

def cohort_lesson_overlap_rule(model, c, t):
    # For each cohort and time slot, sum up all scheduled lessons and ensure it does not exceed 1
    return sum(model.cohort_schedule[c, l, r, t] for l in model.lessons for r in classrooms) <= 1

# Applying the constraint to the model for each cohort and time slot
model.cohort_lesson_overlap = Constraint(model.cohorts, time_slots, rule=cohort_lesson_overlap_rule)




def cohort_lesson_overlap_rule(model, c, t):
    # Ensure each cohort is not scheduled for more than one lesson at the same time slot
    return sum(model.cohort_schedule[c, l, r, t] for l in lessons for r in classrooms) <= 1
model.cohort_lesson_overlap = Constraint(cohorts, time_slots, rule=cohort_lesson_overlap_rule)

# Linking schedule to cohort_schedule
def link_cohort_schedule_rule(model, c, l, r, t):
    if c in lessons_to_cohorts[l]:
        return model.cohort_schedule[c, l, r, t] <= model.schedule[l, r, t]
    else:
        return model.cohort_schedule[c, l, r, t] == 0
model.link_cohort_schedule = Constraint(cohorts, lessons, classrooms, time_slots, rule=link_cohort_schedule_rule)




# --------------------------------

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# # # Step 1: Create a mapping of lessons to cohorts based on your JSON data
# lessons_to_cohorts = {}
# for cohort in cohorts_data['cohorts']:
#     for lesson_code in cohort['lessons']:
#         if lesson_code not in lessons_to_cohorts:
#             lessons_to_cohorts[lesson_code] = []
#         lessons_to_cohorts[lesson_code].append(cohort['name'])

# lessons_to_cohorts = {}
# for cohort in cohorts_data['cohorts']:
#     for lesson_code in cohort['lessons']:
#         if lesson_code not in lessons_to_cohorts:
#             lessons_to_cohorts[lesson_code] = []
#         lessons_to_cohorts[lesson_code].append(cohort['name'])




# Step 2: Aggregate scheduled lessons for each cohort
cohort_schedules = {cohort['name']: {} for cohort in cohorts_data['cohorts']}
for l in model.lessons:
    for r in classrooms:
        for t in time_slots:
            if model.schedule[l, r, t].value == 1:
                lesson_name = lessons[l]['name']
                classroom_name = classroom_names[r]
                time_slot_period = time_slot_periods[t]
                for cohort_name in lessons_to_cohorts.get(l, []):
                    if t not in cohort_schedules[cohort_name]:
                        cohort_schedules[cohort_name][t] = []
                    cohort_schedules[cohort_name][t].append((lesson_name, classroom_name, time_slot_period))

# Step 3: Format and display the output for each cohort
for cohort_name, schedule in cohort_schedules.items():
    print(f"Cohort: {cohort_name}")
    for time_slot, lessons in sorted(schedule.items(), key=lambda x: x[0]):
        for lesson_detail in lessons:
            lesson_name, classroom_name, time_slot_period = lesson_detail
            print(f"{time_slot_period}: {lesson_name}: Classroom: {classroom_name}")
    print("\n" + "-"*40 + "\n")  # Separator for readability



(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().
Cohort: Future Innovators
Monday Morning 09-11: Software Development Lab: Classroom: IT Lab
Monday Morning 09-11: Computer Networks: Classroom: Advanced IT Lab
Wednesday Morning 11-13: Software Development Lab: Classroom: IT Lab
Wednesday Morning 11-13: Computer Networks: Classroom: Advanced IT Lab
Wednesday Afternoon 14-16: Software Development Lab: Classroom: IT Lab
Wednesday Afternoon 16-18: Foundational Mathematics: Classroom: Biology Lab
Wednesday Afternoon 16-18: Data Science with Python: Classroom: IT Lab
Thursday Morning 09-11: Data Science with Python: Classroom: IT Lab
Thursday Morning 11-13: Foundational Mathematics: Classroom: Biology Lab
Thursday Morning 11-13: Computer Networks: Classroom:

In [13]:
from pyomo.environ import *
import json

# Function to load JSON data from a file
def load_json_data(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Load data
classroom_data = load_json_data('Data/classrooms.json')
time_slots_data = load_json_data('Data/time_slots.json')
lessons_data = load_json_data('Data/lessons.json')  # New
teachers_data = load_json_data('Data/teachers.json')  # New
cohorts_data = load_json_data('Data/cohorts.json')  # New

# Model initialization
model = ConcreteModel()

# Existing sets and data structures
classrooms_data = {classroom['code']: classroom['seats'] for classroom in classroom_data['classrooms']}
classrooms = set(classrooms_data.keys())
time_slots = {slot['id'] for slot in time_slots_data['time_slots']}
classroom_names = {classroom['code']: classroom['name'] for classroom in classroom_data['classrooms']}
time_slot_periods = {slot['id']: slot['period'] for slot in time_slots_data['time_slots']}

# New data structures
lessons = {lesson['code']: lesson for lesson in lessons_data['lessons']}
teachers = {teacher['name']: teacher for teacher in teachers_data['teachers']}
cohorts = {cohort['name']: cohort for cohort in cohorts_data['cohorts']}

# Update model sets to include lessons
model.lessons = Set(initialize=lessons.keys())
model.teachers = Set(initialize=teachers.keys())
model.cohorts = Set(initialize=cohorts.keys())

# Assuming each time slot represents 2 hours
time_slot_duration = 2

# Calculate required time slots for each lesson
required_time_slots = {lesson_code: lesson['hours'] // time_slot_duration 
                       for lesson_code, lesson in lessons.items()}


# Variables (updated to include lessons)
# model.schedule = Var(model.lessons, classrooms, time_slots, within=Binary)
# Variables
model.schedule = Var(model.lessons, classrooms, time_slots, within=Binary)
# This variable indicates if a lesson (l) is scheduled for a cohort (c) in a classroom (r) at a time slot (t)
# model.cohort_schedule = Var(model.cohorts, model.lessons, classrooms, time_slots, within=Binary)

# Parameters
required_time_slots = {lesson['code']: lesson['hours'] // 2 for lesson in lessons_data['lessons']}  # Assuming 2 hours per time slot
lessons_to_cohorts = {lesson['code']: [] for lesson in lessons_data['lessons']}

# --------------------------------
# Mapping of cohort names to the lessons they are taking
cohort_lessons_mapping = {cohort: set() for cohort in cohorts}
for lesson_code, cohorts_list in lessons_to_cohorts.items():
    for cohort_name in cohorts_list:
        if cohort_name in cohort_lessons_mapping:
            cohort_lessons_mapping[cohort_name].add(lesson_code)




def lesson_schedule_rule(model, l):
    return sum(model.schedule[l, r, t] for r in classrooms for t in time_slots) >= 1
model.lesson_schedule = Constraint(model.lessons, rule=lesson_schedule_rule)

def unique_lesson_schedule_constraint(model, r, t):
    return sum(model.schedule[l, r, t] for l in model.lessons) <= 1
model.unique_lesson_schedule = Constraint(classrooms, time_slots, rule=unique_lesson_schedule_constraint)

def lesson_time_requirement_rule(model, l):
    return sum(model.schedule[l, r, t] for r in classrooms for t in time_slots) == required_time_slots[l]
model.lesson_time_requirement = Constraint(model.lessons, rule=lesson_time_requirement_rule)

from pyomo.environ import Constraint, ConstraintList, Binary, ConcreteModel, Set, SolverFactory, Var

def correct_classroom_constraint(model, l, r, t):
    lesson_classroom = lessons[l]['classroom']
    # If the lesson requires a specific classroom, check it's scheduled in the correct one
    if lesson_classroom != "any":
        # If the lesson is assigned to an incorrect classroom, enforce the variable to be 0
        return model.schedule[l, r, t] == 0 if r != lesson_classroom else Constraint.Skip
    # If the lesson can be in any classroom, the constraint is always satisfied
    return Constraint.Skip

# Apply the corrected constraint to the model
model.correct_classroom_constraint = Constraint(model.lessons, classrooms, time_slots, rule=correct_classroom_constraint)



from pyomo.environ import Constraint

def cohort_overlap_rule(model, c, t):
    lessons_for_cohort = cohort_lessons_mapping[c]
    # Check if there are any lessons for the cohort to avoid creating an invalid constraint
    if not lessons_for_cohort:
        # If there are no lessons for the cohort, skip this constraint as it's not applicable
        return Constraint.Skip
    # Calculate the sum of scheduled lessons for the cohort in the given time slot
    scheduled_lessons_sum = sum(model.schedule[l, r, t] for l in lessons_for_cohort for r in classrooms)
    # If the expression does not involve any variables, return Constraint.Feasible to avoid the ValueError
    if scheduled_lessons_sum.is_fixed():
        # Check if the fixed value violates the constraint
        if scheduled_lessons_sum() > 1:
            return Constraint.Infeasible
        else:
            return Constraint.Feasible
    else:
        return scheduled_lessons_sum <= 1

# Applying the updated constraint for each cohort and time slot
model.cohort_overlap = Constraint(model.cohorts, time_slots, rule=cohort_overlap_rule)






# --------------------------------

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

lessons_to_cohorts = {}
for cohort in cohorts_data['cohorts']:
    for lesson_code in cohort['lessons']:
        if lesson_code not in lessons_to_cohorts:
            lessons_to_cohorts[lesson_code] = []
        lessons_to_cohorts[lesson_code].append(cohort['name'])

# Step 2: Aggregate scheduled lessons for each cohort
cohort_schedules = {cohort['name']: {} for cohort in cohorts_data['cohorts']}
for l in model.lessons:
    for r in classrooms:
        for t in time_slots:
            if model.schedule[l, r, t].value == 1:
                lesson_name = lessons[l]['name']
                classroom_name = classroom_names[r]
                time_slot_period = time_slot_periods[t]
                for cohort_name in lessons_to_cohorts.get(l, []):
                    if t not in cohort_schedules[cohort_name]:
                        cohort_schedules[cohort_name][t] = []
                    cohort_schedules[cohort_name][t].append((lesson_name, classroom_name, time_slot_period))

# Step 3: Format and display the output for each cohort
for cohort_name, schedule in cohort_schedules.items():
    print(f"Cohort: {cohort_name}")
    for time_slot, lessons in sorted(schedule.items(), key=lambda x: x[0]):
        for lesson_detail in lessons:
            lesson_name, classroom_name, time_slot_period = lesson_detail
            print(f"{time_slot_period}: {lesson_name}: Classroom: {classroom_name}")
    print("\n" + "-"*40 + "\n")  # Separator for readability



Cohort: Future Innovators
Monday Morning 09-11: Software Development Lab: Classroom: IT Lab
Monday Morning 09-11: Computer Networks: Classroom: Advanced IT Lab
Wednesday Morning 11-13: Software Development Lab: Classroom: IT Lab
Wednesday Morning 11-13: Computer Networks: Classroom: Advanced IT Lab
Wednesday Afternoon 14-16: Software Development Lab: Classroom: IT Lab
Wednesday Afternoon 16-18: Foundational Mathematics: Classroom: Biology Lab
Wednesday Afternoon 16-18: Data Science with Python: Classroom: IT Lab
Thursday Morning 09-11: Data Science with Python: Classroom: IT Lab
Thursday Morning 11-13: Foundational Mathematics: Classroom: Biology Lab
Thursday Morning 11-13: Computer Networks: Classroom: Advanced IT Lab
Thursday Morning 11-13: Data Science with Python: Classroom: IT Lab
Thursday Afternoon 14-16: Foundational Mathematics: Classroom: Biology Lab
Thursday Afternoon 14-16: Data Science with Python: Classroom: IT Lab
Thursday Afternoon 16-18: Foundational Mathematics: Classr

In [11]:
results = solver.solve(model, tee=True)  # tee=True will print solver output
# Check solver status
model.solutions.load_from(results)
if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print("Solution is optimal.")
elif results.solver.termination_condition == TerminationCondition.infeasible:
    print("No feasible solution found.")
else:
    print(f"Solver Status: {results.solver.status}, Termination Condition: {results.solver.termination_condition}")


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\Ferna\AppData\Local\Temp\tmph81zl9f9.glpk.raw --wglp C:\Users\Ferna\AppData\Local\Temp\tmpvux6kyfa.glpk.glp
 --cpxlp C:\Users\Ferna\AppData\Local\Temp\tmpuqx4yfie.pyomo.lp
Reading problem data from 'C:\Users\Ferna\AppData\Local\Temp\tmpuqx4yfie.pyomo.lp'...
C:\Users\Ferna\AppData\Local\Temp\tmpuqx4yfie.pyomo.lp:106472: warning: lower bound of variable 'x4' redefined
C:\Users\Ferna\AppData\Local\Temp\tmpuqx4yfie.pyomo.lp:106472: warning: upper bound of variable 'x4' redefined
8780 rows, 18001 columns, 62120 non-zeros
18000 integer variables, all of which are binary
124472 lines were read
Writing problem data to 'C:\Users\Ferna\AppData\Local\Temp\tmpvux6kyfa.glpk.glp'...
89566 lines were written
GLPK Integer Optimizer 5.0
8780 rows, 18001 columns, 62120 non-zeros
18000 integer variables, all of which are binary
Preprocessing...
660 rows, 9880 columns, 29640 non-zeros
9880 integer variables, all o

In [12]:
for c in model.cohorts:
    print(f"Cohort: {c}")
    for t in time_slots:
        for l in lessons:
            for r in classrooms:
                if model.schedule[l, r, t].value == 1:
                    # Make sure this condition is correctly identifying scheduled lessons
                    print(f"    {t}: {lessons[l]['name']} in {classroom_names[r]}")
    print("\n----------------------------------------\n")



Cohort: Future Innovators
    TS18: General Sciences Overview in Biology Lab
    TS18: Introduction to Programming in Art Classroom
    TS18: Machine Learning Basics in Fine Arts Studio
    TS19: Contemporary Art Practices in Biology Lab
    TS19: Introduction to Programming in Art Classroom
    TS19: Machine Learning Basics in Fine Arts Studio
    TS15: Foundational Mathematics in Biology Lab
    TS15: General Sciences Overview in Art Classroom
    TS15: Creative Composition in Fine Arts Studio
    TS15: Data Science with Python in IT Lab
    TS3: Web Technologies in Art Classroom
    TS3: Visual Communication in Biology Lab
    TS3: Environmental Science in Fine Arts Studio
    TS11: Software Development Lab in IT Lab
    TS11: 3D Art and Design in Art Classroom
    TS11: Animation Techniques in Fine Arts Studio
    TS11: Cybersecurity Fundamentals in Advanced IT Lab
    TS11: Architectural Design Fundamentals in Design Lab
    TS11: Game Development Essentials in Chemistry Lab
    T